Brief description of the problem and data

The text discusses the intersection of art and technology, focusing on how generative adversarial networks (GANs) can replicate the distinct styles of artists like Claude Monet. It highlights the advancements in computer vision that enabled the creation of original paintings or enhanced photos in Monet's style. The project encourages exploration of whether these data science techniques can produce artworks that classifiers might mistake for genuine Monet pieces, posing an exciting challenge.

In [ ]:
# import zipfile

# with zipfile.ZipFile('/workspaces/GANs-coursera/data/gan-getting-started.zip', 'r') as zip_ref:
#     zip_ref.extractall('/workspaces/GANs-coursera/data/')

In [2]:
%pip install opencv-contrib-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 48.0 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, random, json, PIL, shutil, re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()


REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')
AUTO = tf.data.experimental.AUTOTUNE

REPLICAS: 1


In [4]:
HEIGHT = 256
WIDTH = 256
CHANNELS = 3
EPOCHS = 50
BATCH_SIZE = 1

In [8]:
monet_jpg_directory = '/workspaces/GANs-coursera/data/gan-getting-started/monet_jpg'
photo_jpg_directory = '/workspaces/GANs-coursera/data/gan-getting-started/'
def getImagePaths(path):
    image_names = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            fullpath = os.path.join(dirname, filename)
            image_names.append(fullpath)
    return image_names
monet_images_path = getImagePaths(monet_jpg_directory)
photo_images_path = getImagePaths(photo_jpg_directory)
print(f"Number of Monet images: {len(monet_images_path)}\n")
print(f"Number of Photo images: {len(photo_images_path)}\n")

Number of Monet images: 300

Number of Photo images: 7364



Number of Monet images: 0

Number of Photo images: 0

